In [ ]:
import requests
import json
import configparser
import time

from pprint import pprint
from os import path
from PIL import Image
import IPython.display as display

In [ ]:
g_debugLog = True

class ALALogger:
    def log(in_log_self, in_str):
        print(f"{in_log_self}: log: {in_str}")
        
    def debugLog(in_log_self, in_str):
        if g_debugLog:
            ALALogger.log(in_log_self, f": debug: {in_str}")
    
class ALAResourceManager:
    def __init__(self, in_ALAMain, in_root_dir, in_data_dragon_dir_rel):
        self._ALAMain = in_ALAMain
        self._root_dir = in_root_dir
        self._data_dragon_dir_rel = in_data_dragon_dir_rel
        self._cache_sprite_champ = {}
        # load champions and championIds map.
        self._champions = {}
        self._id_to_champ = {}
        with open(path.join(self._root_dir, self._data_dragon_dir_rel, f"10.16.1/data/en_US/champion.json"), encoding='utf-8') as f:
            self._champions = json.loads(f.read())['data']
        for ch in self._champions:
            self._id_to_champ[int(self._champions[ch]['key'])] = ch
    
    def getProfileIconImage(self, in_profile_icon_id):
        return Image.open(path.join(self._root_dir, self._data_dragon_dir_rel, f"10.16.1/img/profileicon/{in_profile_icon_id}.png"))
    
    def getRankEmblem(self, in_tier):
        return Image.open(path.join(self._root_dir, f"Assets/ranked-emblems/Emblem_{(in_tier).capitalize()}.png"))
    
    def getChampionName(self, in_champion_id):
        return self._id_to_champ[in_champion_id]
    
    def getChampionIcon(self, in_champion_id):
        champ_name = self.getChampionName(in_champion_id)
        sprite_file_name = self._champions[champ_name]['image']['sprite']
        if not sprite_file_name in self._cache_sprite_champ:
            self._cache_sprite_champ[sprite_file_name] = Image.open(path.join(self._root_dir, self._data_dragon_dir_rel, f"10.16.1/img/sprite/{sprite_file_name}"))
        return self._cache_sprite_champ[sprite_file_name].crop((self._champions[champ_name]['image']['x'],
                                                                self._champions[champ_name]['image']['y'],
                                                                self._champions[champ_name]['image']['x'] + self._champions[champ_name]['image']['w'],
                                                                self._champions[champ_name]['image']['y'] + self._champions[champ_name]['image']['h']
                                                              ))

class RiotAPICaller:
    def __init__(self, in_ALAMain, in_API_Key):
        self._ALAMain = in_ALAMain
        self._API_KEY = in_API_Key
        self._max_gateway_timeout_attempts = 10
        self._gateway_timeout_attempts = 0
        self._time_last_99_requests = [ (time.time() - i*60) for i in range(99) ]
        
    def runGetSubdomain(self, in_subdomain_with_params, in_region, in_additional_params_str = ''):
        # Prepare the URL.
        get_url = f"https://{in_region}.api.riotgames.com/lol/{in_subdomain_with_params}?api_key={self._API_KEY}{in_additional_params_str}"
        req_done = False
        while not req_done:
            # Wait for time rate.
            #     20 reqs every 1 second:
            if 1.0 >= time.time() - self._time_last_99_requests[18]:
                ALALogger.log(self,f"waiting for 1 second.")
                time.sleep(1)
            #    100 reqs every 2 minutes:
            if 120.0 >= time.time() - self._time_last_99_requests[98]:
                wait_time = self._time_last_99_requests[98] + 121.0 - time.time()
                ALALogger.log(self,f"waiting for {wait_time} seconds")
                time.sleep(wait_time)
            # Set time of this request.
            self._time_last_99_requests.pop(-1)
            self._time_last_99_requests.insert(0, time.time())
            # Send request.
            get_response = requests.get(get_url)
            # Verify the response and return data.
            ALALogger.debugLog(self, f"request:{get_url} returned with status code: {get_response.status_code}")
            if get_response.status_code == 200:
                req_done = True
            elif get_response.status_code == 504:
                ALALogger.log(self, f"request:{get_url} returned with status code: {get_response.status_code} (Gateway timeout)")
                gateway_timeout_attempts += 1
                if self._gateway_timeout_attempts > self._max_gateway_timeout_attempts:
                    req_done = True
            else:
                ALALogger.log(self, f"request:{get_url} returned with status code: {get_response.status_code}")
                req_done = True
        if get_response.status_code != 200:
            ALALogger.log(self, f"request:{get_url} returned with status code: {get_response.status_code}")
            raise ValueError(f"request:{get_url} returned with status code: {get_response.status_code}")
        return get_response.json()
    
    # RiotAPI: Champion Mastery
    def runGetChampionMasteriesBySummonerId(self, in_encrypted_summoner_id, in_region):
        subdomain_with_params = f"champion-mastery/v4/champion-masteries/by-summoner/{in_encrypted_summoner_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    # RiotAPI: League
    def runGetSummonerLeagueBySummonerId(self, in_encrypted_summoner_id, in_region):
        subdomain_with_params = f"league/v4/entries/by-summoner/{in_encrypted_summoner_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    def runGetLeagueEntries(self, in_queue, in_tier, in_division, in_page, in_region):
        subdomain_with_params = f"league/v4/entries/{in_queue}/{in_tier}/{in_division}"
        return self.runGetSubdomain(subdomain_with_params, in_region, f"&page={in_page}")
    
    def runGetMasterLeagueEntries(self, in_queue, in_region):
        subdomain_with_params = f"league/v4/masterleagues/by-queue/{in_queue}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    def runGetGrandMasterLeagueEntries(self, in_queue, in_region):
        subdomain_with_params = f"league/v4/grandmasterleagues/by-queue/{in_queue}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    def runGetChallengerLeagueEntries(self, in_queue, in_region):
        subdomain_with_params = f"league/v4/challengerleagues/by-queue/{in_queue}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    # RiotAPI: Match
    def runGetMatchListByAccount(self, in_encrypted_account_id, in_region):
        subdomain_with_params = f"match/v4/matchlists/by-account/{in_encrypted_account_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)

    def runGetMatchInfoByMatchId(self, in_match_id, in_region):
        subdomain_with_params = f"match/v4/matches/{in_match_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)

    def runGetMatchTimelineDataByMatchID(self, in_match_id, in_region):
        subdomain_with_params = f"match/v4/timelines/by-match/{in_match_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    # RiotAPI: Summoner
    def runGetSummonerByName(self, in_summoner_name, in_region):
        subdomain_with_params = f"summoner/v4/summoners/by-name/{in_summoner_name}"
        return self.runGetSubdomain(subdomain_with_params, in_region)

class ChampionInfo:
    def __init__(self, in_ALAMain, in_champion_id):
        self._ALAMain = in_ALAMain
        self._champion_id = in_champion_id
        
    def getName(self):
        return self._ALAMain._resource_man.getChampionName(self._champion_id)
    
    def getIcon(self):
        return self._ALAMain._resource_man.getChampionIcon(self._champion_id)
        
        
class ALAMain:
    def __init__(self):
        # Read config file to get data locations.
        config = configparser.ConfigParser()
        config.read('config.ini')
        self._root_dir = config['DEFAULT']['root_dir']
        riot_api_key_file_rel = config['DEFAULT']['riot_api_key_file_rel']
        self._data_dragon_dir_rel = config['DEFAULT']['data_dragon_dir_rel']
        
        # Load RiotAPI key from file.
        with open(path.join(self._root_dir, riot_api_key_file_rel)) as f:
            self._API_KEY = json.loads(f.read())["API_KEY"]
        
        self._riot_api_caller = RiotAPICaller(self, self._API_KEY)
        self._resource_man = ALAResourceManager(self, self._root_dir, self._data_dragon_dir_rel)
        
    def getSummonerInfo(self, in_summoner_name, in_region):
        summoner = self._riot_api_caller.runGetSummonerByName(in_summoner_name, in_region)
        if summoner:
            img_prof_icon = self._resource_man.getProfileIconImage(summoner['profileIconId'])
            display.display(img_prof_icon.resize((50,50)))
            print(f"Summoner: {summoner['name']} (level:{summoner['summonerLevel']})")
            summoner_league = self._riot_api_caller.runGetSummonerLeagueBySummonerId(summoner['id'], in_region)
            if summoner_league:
                for lidx in range(len(summoner_league)):
                    img_tier = self._resource_man.getRankEmblem(summoner_league[lidx]['tier'])
                    display.display(img_tier.resize((50,50)))
                    print(f"Queue: {summoner_league[lidx]['queueType']} Rank: {summoner_league[lidx]['tier']}-{summoner_league[lidx]['rank']}")
    
    def getChampionInfo(self, in_champion_id):
        return ChampionInfo(self, in_champion_id)
    
    def collectHighTierFlexAndClashMatchData(self, in_region):
        # Valid queueIds
        FlexAndClashQueueIds = [440, 700]
        # collect summonner_name list of high elo players in Flex Queue
        entry_list = self._riot_api_caller.runGetChallengerLeagueEntries('RANKED_FLEX_SR',in_region)['entries']
        summoner_name_list = [ entry['summonerName'] for entry in entry_list]
        # collect the set of gameIds of those players which are in Flex Queue
        match_id_set = set()
        for summoner_name in summoner_name_list:
            summoner_info = self._riot_api_caller.runGetSummonerByName(summoner_name, in_region)
            summoner_match_list = self._riot_api_caller.runGetMatchListByAccount(summoner_info['accountId'], in_region)
            match_id_set.update([ match['gameId'] for match in summoner_match_list['matches'] if match['queue'] in FlexAndClashQueueIds])
        with open('match_id_set.json','w') as f:
            json.dump(match_id_set,f)
        # collect the match information
        match_list = []
        for match_id in match_id_set:
            match_info = self._riot_api_caller.runGetMatchInfoByMatchId(match_id, in_region)
            match_list.append(match_info)
        with open('match_list.json','w') as f:
            json.dump(match_list,f)
        return match_list
    

In [ ]:
ala = ALAMain()
ala.getSummonerInfo('deamonpog','na1')
entries = ala.collectHighTierFlexAndClashMatchData('euw1')

In [ ]:
pprint(len(entries))
pprint(entries[0].keys())
[ display.display(ala.getChampionInfo(p['championId']).getIcon()) for p in entries[0]['participants'] ]
[ ala.getChampionInfo(p['championId']).getName() for p in entries[0]['participants'] ]

In [ ]:
# with open(path.join(ala._root_dir, f"Assets/queues.json"), encoding='utf-8') as f:
#     qs = json.loads(f.read())
# pprint([ q for q in qs if ('Rank' in str(q['description'])) ])

import pandas as pd

df = pd.read_json(path.join(ala._root_dir, f"Assets/queues.json"))
df[df.map == "Summoner's Rift"]

In [ ]:
x = [i for i in range(10)]
print(x)
x.pop(-1)
# x.insert(0,10)
print(x)

In [ ]:
in_sumName = 'Anduru'
in_region = 'euw1'

get_response = runGetSummonerByName(in_sumName,in_region)
if get_response.status_code == 200:
    summoner_info = get_response.json()
    img_prof_icon = Image.open(path.join(root_dir, data_dragon_root, f"10.16.1/img/profileicon/{summoner_info['profileIconId']}.png"))
    display.display(img_prof_icon.resize((50,50)))
    print(f"Summoner: {summoner_info['name']} (level:{summoner_info['summonerLevel']})")
else:
    print(f"Response code: {get_response.status_code}")

get_response = runGetSummonerLeagueBySummonerId(summoner_info['id'],in_region)
if get_response.status_code == 200:
    summoner_league = get_response.json()
    for lidx in range(len(summoner_league)):
        img_tier = Image.open(path.join(root_dir, f"Assets/ranked-emblems/Emblem_{(summoner_league[lidx]['tier']).capitalize()}.png"))
        display.display(img_tier.resize((50,50)))
        print(f"Queue: {summoner_league[lidx]['queueType']} Rank: {summoner_league[lidx]['tier']}-{summoner_league[lidx]['rank']}")
else:
    print(f"Response code: {get_response.status_code}")
    
get_response = runGetChampionMasteriesBySummonerId(summoner_info['id'],in_region)
if get_response.status_code == 200:
    summoner_mastery = get_response.json()
    for chidx in range(len(summoner_mastery)):
        ch = id_to_champ[summoner_mastery[chidx]['championId']]
        img_champion = Image.open(path.join(root_dir, data_dragon_root, f"10.16.1/img/sprite/{champ_data[ch]['image']['sprite']}"))
        display.display(img_champion.crop((champ_data[ch]['image']['x'],champ_data[ch]['image']['y'],champ_data[ch]['image']['x'] + champ_data[ch]['image']['w'],champ_data[ch]['image']['y'] + champ_data[ch]['image']['h'])).resize((50,50)))
        print(f"Mastery: {summoner_mastery[chidx]['championLevel']} with Mastery Points: {summoner_mastery[chidx]['championPoints']}")
        if chidx >= 10:
            break
else:
    print(f"Response code: {get_response.status_code}")
    
get_response = runGetMatchListByAccount(summoner_info['accountId'],'euw1')
if get_response.status_code == 200:
    match_list = get_response.json()
    print(f"Number of matches: {len(match_list['matches'])}")
else:
    print(f"Response code: {get_response.status_code}")
          
get_response = runGetMatchInfoByMatchId(match_list['matches'][0]['gameId'],'euw1')
if get_response.status_code == 200:
    match_info = get_response.json()
    print(f"The Game mode: {match_info['gameMode']}")
else:
    print(f"Response code: {get_response.status_code}")

get_response = runGetMatchTimelineDataByGameID(match_list['matches'][0]['gameId'],'euw1')
if get_response.status_code == 200:
    match_data = get_response.json()
    print(f"Number of frames: {len(match_data['frames'])}")
else:
    print(f"Response code: {get_response.status_code}")